In [137]:
import http.client
import config
import pandas as pd
import json
import datetime
import os
import folium

In [138]:
conn1 = http.client.HTTPSConnection("app.goflightlabs.com")
conn2 = http.client.HTTPSConnection("app.goflightlabs.com")

conn1.request("GET", f"/flights?access_key={config.api_key}&arrIata=ATL")
conn2.request("GET", f"/flights?access_key={config.api_key}&depIata=ATL")

real_time_arrivals = conn1.getresponse().read().decode('utf-8')
real_time_arrivals = json.loads(real_time_arrivals)

real_time_departures = conn2.getresponse().read().decode('utf-8')
real_time_departures = json.loads(real_time_departures)

In [139]:
df_real_time_departures = pd.json_normalize(real_time_departures['data'])
df_real_time_arrivals = pd.json_normalize(real_time_arrivals['data'])
df_real_time_flights = pd.concat([df_real_time_departures, df_real_time_arrivals])

In [140]:
df_real_time_flights

,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,arrival.iataCode,arrival.icaoCode,departure.iataCode,...,flight.number,geography.altitude,geography.direction,geography.latitude,geography.longitude,speed.horizontal,speed.isGround,speed.vspeed,system.squawk,system.updated
0,en-route,B752,A8DF06,B752,N6707A,DL,DAL,SNA,KSNA,ATL,...,447,9753.60,260,34.03,-115.74,885.256,0,0,None,1684436790
1,en-route,B753,A77AE0,B753,N581NW,DL,DAL,LAX,KLAX,ATL,...,895,10363.20,266,34.55,-103.04,814.880,0,0,None,1684437064
2,en-route,B752,A8DE56,B752,N6702,DL,DAL,FLL,KFLL,ATL,...,1375,6416.04,169,33.08,-84.19,700.056,0,0,None,1684436785
3,landed,A321,A446B6,A321,N375DN,DL,DAL,PHX,KPHX,ATL,...,796,0.00,137,33.44,-112.02,31.484,1,0,None,1684437074
4,en-route,A321,A41F37,A321,N365DN,DL,DAL,PBI,KPBI,ATL,...,1173,10660.40,145,29.80,-82.73,853.772,0,0,None,1684436786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,en-route,B77W,06A078,B77W,A7-BAK,QR,QTR,ATL,KATL,DOH,...,755,10668.00,155,36.30,-83.36,851.920,0,0,None,1684436480
158,en-route,E55P,A390D5,E55P,N329MC,,,ATL,KATL,PSF,...,XXF,12496.80,226,39.31,-76.97,679.684,0,0,None,1684436764
159,en-route,B763,A49CE2,B763,N397AZ,,,ATL,KATL,CVG,...,3434,8199.12,176,38.25,-84.49,783.396,0,0,None,1684436480
160,en-route,A320,A81B2A,A320,N621NK,NK,NKS,ATL,KATL,ORD,...,1784,6553.20,178,41.22,-87.89,733.392,0,0,None,1684436469


In [141]:
df_airports = pd.read_csv('./data/airports.csv')

In [142]:
df_airports_arrival = df_airports.rename(columns={
    'airportId': 'arrival.airportId',
    'codeIataAirport': 'arrival.iataCode',
    'codeIataCity': 'arrival.codeIataCity',
    'codeIcaoAirport': 'arrival.codeIcaoAirport',
    'codeIso2Country': 'arrival.codeIso2Country',
    'latitudeAirport': 'arrival.latitudeAirport',
    'longitudeAirport': 'arrival.longitudeAirport',
    'nameAirport': 'arrival.nameAirport',
    'nameCountry': 'arrival.nameCountry',
    'timezone': 'arrival.timezone'
})

df_airports_arrival = df_airports_arrival.drop(columns=['GMT', 'geonameId','phone'])


In [143]:
df_real_time_flights = pd.merge(df_real_time_flights, df_airports_arrival, how='left', on='arrival.iataCode')

In [144]:
df_airports_departure = df_airports.rename(columns={
    'airportId': 'departure.airportId',
    'codeIataAirport': 'departure.iataCode',
    'codeIataCity': 'departure.codeIataCity',
    'codeIcaoAirport': 'departure.codeIcaoAirport',
    'codeIso2Country': 'departure.codeIso2Country',
    'latitudeAirport': 'departure.latitudeAirport',
    'longitudeAirport': 'departure.longitudeAirport',
    'nameAirport': 'departure.nameAirport',
    'nameCountry': 'departure.nameCountry',
    'timezone': 'departure.timezone'
})
df_airports_departure = df_airports_departure.drop(columns=['GMT', 'geonameId','phone'])


In [145]:
df_airports_departure

,departure.airportId,departure.iataCode,departure.codeIataCity,departure.codeIcaoAirport,departure.codeIso2Country,departure.latitudeAirport,departure.longitudeAirport,departure.nameAirport,departure.nameCountry,departure.timezone
0,1,AAA,AAA,NTGA,PF,-17.050000,-145.416670,Anaa,French Polynesia,Pacific/Tahiti
1,2,AAB,AAB,YARY,AU,-26.700000,141.041670,Arrabury,Australia,Australia/Brisbane
2,3,AAC,AAC,HEAR,EG,31.133333,33.750000,El Arish International Airport,Egypt,Africa/Cairo
3,4,AAE,AAE,DABB,DZ,36.821392,7.811857,Les Salines,NaN,Africa/Algiers
4,5,AAF,AAF,KAAF,US,29.733334,-84.983330,Apalachicola Regional,United States,America/New_York
...,...,...,...,...,...,...,...,...,...,...
10279,10566,VKX,NaN,KVKX,US,38.748642,-76.955922,Potomac Airfield,United States,NaN
10280,10567,WRN,NaN,YWDG,AU,-30.031667,119.390000,Windarling Airport,Australia,NaN
10281,10568,WXF,NaN,EGVT,GB,51.971901,0.509444,Wethersfield Airport,United Kingdom,NaN
10282,10569,XCD,NaN,LFLH,FR,46.826111,4.817500,Chalon – Champforgeuil Airfield,France,NaN


In [146]:
df_real_time_flights = pd.merge(df_real_time_flights, df_airports_departure, how='left', on='departure.iataCode')

In [147]:
df_real_time_flights

,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,arrival.iataCode,arrival.icaoCode,departure.iataCode,...,arrival.timezone,departure.airportId,departure.codeIataCity,departure.codeIcaoAirport,departure.codeIso2Country,departure.latitudeAirport,departure.longitudeAirport,departure.nameAirport,departure.nameCountry,departure.timezone
0,en-route,B752,A8DF06,B752,N6707A,DL,DAL,SNA,KSNA,ATL,...,America/Los_Angeles,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
1,en-route,B753,A77AE0,B753,N581NW,DL,DAL,LAX,KLAX,ATL,...,America/Los_Angeles,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
2,en-route,B752,A8DE56,B752,N6702,DL,DAL,FLL,KFLL,ATL,...,America/New_York,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
3,landed,A321,A446B6,A321,N375DN,DL,DAL,PHX,KPHX,ATL,...,America/Phoenix,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
4,en-route,A321,A41F37,A321,N365DN,DL,DAL,PBI,KPBI,ATL,...,America/New_York,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,en-route,B77W,06A078,B77W,A7-BAK,QR,QTR,ATL,KATL,DOH,...,America/New_York,1875,DOH,OTBD,QA,25.267569,51.558067,Doha International,Qatar,Asia/Qatar
264,en-route,E55P,A390D5,E55P,N329MC,,,ATL,KATL,PSF,...,America/New_York,6315,PSF,NaN,US,42.426666,-73.291664,Pittsfield,United States,America/New_York
265,en-route,B763,A49CE2,B763,N397AZ,,,ATL,KATL,CVG,...,America/New_York,1604,CVG,KCVG,US,39.055500,-84.661450,Cincinnati/Northern Kentucky,United States,America/New_York
266,en-route,A320,A81B2A,A320,N621NK,NK,NKS,ATL,KATL,ORD,...,America/New_York,5818,CHI,KORD,US,41.976913,-87.904880,Chicago O'hare International,United States,America/Chicago


In [148]:
df_airlines = pd.read_csv('./data/airlines.csv')
df_airlines = df_airlines.rename(columns={
    'codeIataAirline': 'airline.iataCode',
    'codeIcaoAirline': 'airline.icaoCode'})
df_airlines = df_airlines.dropna(subset=['airline.iataCode', 'airline.icaoCode'])
df_airlines

,ageFleet,airlineId,callsign,codeHub,airline.iataCode,airline.icaoCode,codeIso2Country,founding,iataPrefixAccounting,nameAirline,nameCountry,sizeAirline,statusAirline,type
0,10.9,1,AMERICAN,DFW,AA,AAL,US,1934,1.0,American Airlines,United States,963,active,scheduled
1,17.0,2,DELTA,ATL,DL,DAL,US,1928,6.0,Delta Air Lines,United States,823,active,"scheduled,division"
2,13.8,3,UNITED,ORD,UA,UAL,US,1931,16.0,United Airlines,United States,715,active,"scheduled,division"
3,12.3,4,SOUTHWEST,DAL,WN,SWA,UM,1967,526.0,Southwest Airlines Co.,United States Minor Outlying Islands,712,active,scheduled
4,6.9,5,CHINA SOUTHERN,CAN,CZ,CSN,CN,2004,784.0,China Southern Airlines,China,498,active,scheduled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,0.0,3957,NaN,YQB,ZX,AAQ,CA,0,NaN,Air Alliance (Canada),Canada,0,merged,scheduled
3956,0.0,3958,AIRCOACH,NaN,ZX,ABL,CA,0,742.0,Air B.C.,Canada,0,historical,NaN
3957,0.0,3959,FLYZOOM,GLA,ZX,UKZ,UK,0,NaN,Zoom Airlines,United Kingdom,0,historical,charter
3958,0.0,3960,ADA AIR,TIA,ZY,ADE,AL,0,121.0,Ada Air,Albania,0,historical,scheduled


In [149]:
df_real_time_flights = pd.merge(df_real_time_flights, df_airlines, how='left',
                                left_on=['airline.iataCode', 'airline.icaoCode'],
                                right_on=['airline.iataCode', 'airline.icaoCode'])

now = datetime.datetime.now()

# Format date and time as a string to be used in the filename
datetime_string = now.strftime("%Y-%m-%d %H:%M:%S")
df_real_time_flights['date_EST'] = datetime_string

In [150]:
df_real_time_flights

,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,arrival.iataCode,arrival.icaoCode,departure.iataCode,...,codeHub,codeIso2Country,founding,iataPrefixAccounting,nameAirline,nameCountry,sizeAirline,statusAirline,type,date_EST
0,en-route,B752,A8DF06,B752,N6707A,DL,DAL,SNA,KSNA,ATL,...,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division",2023-05-18 15:24:12
1,en-route,B753,A77AE0,B753,N581NW,DL,DAL,LAX,KLAX,ATL,...,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division",2023-05-18 15:24:12
2,en-route,B752,A8DE56,B752,N6702,DL,DAL,FLL,KFLL,ATL,...,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division",2023-05-18 15:24:12
3,landed,A321,A446B6,A321,N375DN,DL,DAL,PHX,KPHX,ATL,...,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division",2023-05-18 15:24:12
4,en-route,A321,A41F37,A321,N365DN,DL,DAL,PBI,KPBI,ATL,...,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division",2023-05-18 15:24:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,en-route,B77W,06A078,B77W,A7-BAK,QR,QTR,ATL,KATL,DOH,...,DOH,QA,1994.0,157.0,Qatar Airways,Qatar,176.0,active,scheduled,2023-05-18 15:24:12
264,en-route,E55P,A390D5,E55P,N329MC,,,ATL,KATL,PSF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-18 15:24:12
265,en-route,B763,A49CE2,B763,N397AZ,,,ATL,KATL,CVG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-18 15:24:12
266,en-route,A320,A81B2A,A320,N621NK,NK,NKS,ATL,KATL,ORD,...,FLL,UM,1980.0,487.0,Spirit Airlines,United States Minor Outlying Islands,84.0,active,scheduled,2023-05-18 15:24:12


In [151]:
df_real_time_flights = df_real_time_flights.drop(columns=['speed.vspeed', 'system.squawk', 'system.updated', 'ageFleet', 'founding', 'iataPrefixAccounting', 'type'])


df_real_time_flights = df_real_time_flights[['date_EST',
                                           'status',
                                           'aircraft.iataCode',
                                           'aircraft.icao24',
                                           'aircraft.icaoCode',
                                           'aircraft.regNumber',
                                           'airline.iataCode',
                                           'airline.icaoCode',
                                           'nameAirline',
                                           'flight.iataNumber',
                                           'flight.icaoNumber',
                                           'geography.altitude',
                                           'geography.direction',
                                           'geography.latitude',
                                           'geography.longitude',
                                           'speed.horizontal',
                                           'speed.isGround',
                                           'departure.airportId',
                                           'departure.nameAirport',
                                           'departure.iataCode',
                                           'departure.icaoCode',
                                           'departure.codeIataCity',
                                           'departure.codeIso2Country',
                                           'departure.nameCountry',
                                           'departure.latitudeAirport',
                                           'departure.longitudeAirport',
                                           'departure.timezone',
                                           'arrival.airportId',
                                           'arrival.nameAirport',
                                           'arrival.iataCode',
                                           'arrival.icaoCode',
                                           'arrival.codeIataCity',
                                           'arrival.codeIso2Country',
                                           'arrival.nameCountry',
                                           'arrival.latitudeAirport',
                                           'arrival.longitudeAirport',
                                           'arrival.timezone']]


In [152]:
df_real_time_flights

,date_EST,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,nameAirline,flight.iataNumber,...,arrival.airportId,arrival.nameAirport,arrival.iataCode,arrival.icaoCode,arrival.codeIataCity,arrival.codeIso2Country,arrival.nameCountry,arrival.latitudeAirport,arrival.longitudeAirport,arrival.timezone
0,2023-05-18 15:24:12,en-route,B752,A8DF06,B752,N6707A,DL,DAL,Delta Air Lines,DL447,...,7224.0,John Wayne,SNA,KSNA,SNA,US,United States,33.680202,-117.860535,America/Los_Angeles
1,2023-05-18 15:24:12,en-route,B753,A77AE0,B753,N581NW,DL,DAL,Delta Air Lines,DL895,...,4189.0,Los Angeles International,LAX,KLAX,LAX,US,United States,33.943398,-118.408280,America/Los_Angeles
2,2023-05-18 15:24:12,en-route,B752,A8DE56,B752,N6702,DL,DAL,Delta Air Lines,DL1375,...,2344.0,Fort Lauderdale–Hollywood International,FLL,KFLL,FLL,US,United States,26.071491,-80.144905,America/New_York
3,2023-05-18 15:24:12,landed,A321,A446B6,A321,N375DN,DL,DAL,Delta Air Lines,DL796,...,6100.0,Sky Harbor International,PHX,KPHX,PHX,US,United States,33.435036,-112.000160,America/Phoenix
4,2023-05-18 15:24:12,en-route,A321,A41F37,A321,N365DN,DL,DAL,Delta Air Lines,DL1173,...,5966.0,Palm Beach International Airport,PBI,KPBI,PBI,US,United States,26.688885,-80.090440,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,2023-05-18 15:24:12,en-route,B77W,06A078,B77W,A7-BAK,QR,QTR,Qatar Airways,QR755,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York
264,2023-05-18 15:24:12,en-route,E55P,A390D5,E55P,N329MC,,,NaN,XXD,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York
265,2023-05-18 15:24:12,en-route,B763,A49CE2,B763,N397AZ,,,NaN,8C3434,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York
266,2023-05-18 15:24:12,en-route,A320,A81B2A,A320,N621NK,NK,NKS,Spirit Airlines,NK1784,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York


In [153]:
# Get current date and time
now = datetime.datetime.now()

# Format date and time as a string to be used in the filename
datetime_string = now.strftime("%Y-%m-%d_%H-%M-%S")

year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

dir_path = f'./data/real_time/{year}/{month}/{day}/'
os.makedirs(dir_path, exist_ok=True)
# Create filename with date and time
filename = f'{dir_path}{datetime_string}_df_real_time_flights.csv'

# Export DataFrame to CSV
df_real_time_flights.to_csv(filename, index=False)
